In [75]:
import json
import pandas as pd
import mappers
from datetime import datetime
import os

prefix = 'AFE'

directory_path = 'data/pixie/Indoor/Participant_1/'

files_starting_with_prefix = [filename for filename in os.listdir(directory_path) if filename.startswith(prefix)]


dfs = []
df = None

for ele in files_starting_with_prefix:
    current_path = directory_path + ele
    cur_df = mappers.afe_to_df(current_path)
    dfs.append(cur_df)
    if df is None:
        df = cur_df
    else:
        df = pd.concat([df, cur_df])






In [76]:
print(df.columns)


Index(['afe_left_t', 'afe_left_ticktime', 'afe_left_timestamp',
       'afe_left_status', 'afe_left_counter', 'afe_left_m_0_0',
       'afe_left_m_0_1', 'afe_left_m_0_2', 'afe_left_m_0_3', 'afe_left_m_0_4',
       'afe_left_m_0_5', 'afe_right_t', 'afe_right_ticktime',
       'afe_right_timestamp', 'afe_right_status', 'afe_right_counter',
       'afe_right_m_0_0', 'afe_right_m_0_1', 'afe_right_m_0_2',
       'afe_right_m_0_3', 'afe_right_m_0_4', 'afe_right_m_0_5',
       'afe_right_m_0_6', 'afe_right_m_0_7', 'battery_voltage',
       'battery_percent', 'auxSensors_lightAmbient_t',
       'auxSensors_lightAmbient_v_0', 'auxSensors_lightAmbient_v_1',
       'auxSensors_lightAmbient_v_2', 'auxSensors_lightAmbient_v_3',
       'auxSensors_lightAmbient_i_0', 'auxSensors_lightAmbient_i_1',
       'auxSensors_lightAmbient_i_2', 'auxSensors_lightAmbient_i_3',
       'auxSensors_lightAmbient_status_0', 'auxSensors_lightAmbient_status_1',
       'auxSensors_lightAmbient_status_2', 'auxSensors_lig

In [77]:
print(df.head())

  afe_left_t  afe_left_ticktime  afe_left_timestamp  afe_left_status  \
0          L            3715918    1699001623169289                0   
1          L            3715935    1699001623185984                0   
2          L            3715952    1699001623202679                0   
3          L            3715968    1699001623219313                0   
4          L            3715985    1699001623236008                0   

   afe_left_counter  afe_left_m_0_0  afe_left_m_0_1  afe_left_m_0_2  \
0             29814           19167            6302            9952   
1             29815           18921            6379           10077   
2             29816           20336            6490           10175   
3             29817           19470            6470           10321   
4             29818           18126            6451           10424   

   afe_left_m_0_3  afe_left_m_0_4  ...  auxSensors_tempEt_v_right_3  \
0           15459           16930  ...                    27.617188  

In [78]:
first_timestamp = df['afe_left_timestamp']
last_timestamp = df['afe_left_timestamp'].iloc[-1].item()
date_start =  datetime.utcfromtimestamp(first_timestamp / 1e6)
date_end = datetime.utcfromtimestamp(last_timestamp / 1e6)

interv =  date_end - date_start

print(date_start)
print(interv)


2023-11-03 08:53:43.169289
0:16:04.183470


In [79]:
#Finding first blink timestamp
blinks = df[df['blinks_left_peakTime'].notna()]


df['datetime_left'] = pd.to_datetime(df['afe_left_timestamp'], unit='us')
df['datetime_right'] = pd.to_datetime(df['afe_right_timestamp'], unit='us')
df['diff_both_eyes'] = (df['afe_left_timestamp'] - df['afe_right_timestamp']) // 1000
df['diff_left'] = df['datetime_left'].diff()


stats_diff = df['diff_both_eyes'].describe()

print(stats_diff)


count    57649.000000
mean        -4.999358
std          0.026661
min         -5.000000
25%         -5.000000
50%         -5.000000
75%         -5.000000
max         -3.000000
Name: diff_both_eyes, dtype: float64


In [86]:
blinks = df[df['blinks_left_peakTime'].notna()]

print(blinks.head())

df['left_blink_diff'] = df['blinks_left_endTime'] - df['blinks_left_beginTime']
describe_l_blink_diff = df['left_blink_diff'].describe()
print(describe_l_blink_diff)


start = df['datetime_left'].iloc[0]
end = df['datetime_left'].iloc[-1]

for i,time_s in blinks.iterrows():
    time_ms = time_s['datetime_left']    
    difference = time_ms - start
    print(f'Time before blink idx {i}: {difference}')



    afe_left_t  afe_left_ticktime  afe_left_timestamp  afe_left_status  \
5            L            3716002    1699001623252825                0   
79           L            3717237    1699001624488203                0   
94           L            3717487    1699001624738600                0   
225          L            3719674    1699001626924757                0   
270          L            3720425    1699001627676097                0   

     afe_left_counter  afe_left_m_0_0  afe_left_m_0_1  afe_left_m_0_2  \
5               29819           17612            6446           10502   
79              29893           21845            6963           10829   
94              29908           16947            5998            9398   
225             30039           18103            6467           10419   
270             30084           17299            6003            9725   

     afe_left_m_0_3  afe_left_m_0_4  ...  blinks_left_endTime  \
5             14977           16646  ...         1.